# Scenario

on other words testing the performance

In [2]:
import pandas as pd
from llamaindex import search_by_llamaindex, initialize_llamaindex_BGE, initialize_llamaindex_LABSE
import json

## load models

In [3]:
query_engine1 = initialize_llamaindex_BGE()
query_engine2 = initialize_llamaindex_LABSE()

LLM is explicitly disabled. Using MockLLM.
llamaindex: loading bge index from storage
LLM is explicitly disabled. Using MockLLM.


/Users/tijsvandenheuvel/.virtualenvs/doc_retrieval_env/lib/python3.9/site-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  to

llamaindex: loading labse index from storage


In [27]:
query="corruption"
response = query_engine1.query(query)

In [28]:
query="corruption"
response = query_engine2.query(query)

## Load queries

In [4]:
file_name = "../documents/20241029_Scenario Update.xlsx"
query_column_name = "vraag"

df = pd.read_excel(file_name)

queries = df[query_column_name].dropna().tolist()

In [5]:
print(queries)

[' When does aggregation of data qualify as anonymisation under GDPR ?', 'Wanneer komt aggregatie van gegevens in aanmerking als anonimisering in het kader van de AVG?', 'aggregation and anonymisation and gdpr', 'aggregatie en anonimisering en avg', 'Are appraisal documents by HR in scope of the right to access (DSAR) under GDPR ?', 'Zijn beoordelingsdocumenten door HR onderworpen aan  het recht op toegang onder de AVG?', 'HR "appraisal documents" and DSAR', 'HR beoordelingsdocumenten and "recht op toegang" and AVG ', 'What are elements of valid consent?', 'Wat zijn de elementen van een geldige toestemming?', '"elements of valid consent" / "valid consent" and elements', '"elementen van een geldige toestemming" / "geldige toestemming" and elementen', 'Which criteria can help the European Data Protection Authorities to handle complaints relating to de-listing certain links to information affecting data protection?', 'Welke criteria kunnen de Europese gegevensbeschermingsautoriteiten help

## Run queries

In [9]:
structured_results = []

for query in queries:
            k_search_results = search_by_llamaindex(query, query_engine1, "llamaindex_bge")
            print(k_search_results)
            k_titles = [hit["title"] for hit in k_search_results]
            
            v_search_results = search_by_llamaindex(query, query_engine2, "llamaindex_labse")
            v_titles = [hit["title"] for hit in v_search_results]
            
            # Ensure exactly 10 results for each (pad or truncate if necessary)
            k_titles = k_titles[:10] + [""] * (10 - len(k_titles))  # Truncate or pad to 10
            v_titles = v_titles[:10] + [""] * (10 - len(v_titles))  # Truncate or pad to 10
            
            print(len(k_titles),len(v_titles))

            # Add the query row
            structured_results.append({
                "Query": query,
                "search by keyword": k_titles[0] if k_titles else "",
                "search by vector": v_titles[0] if v_titles else "",
            })


            for i in range(1, 10):
                structured_results.append({
                    "Query": "",
                    "search by keyword": k_titles[i] if i < len(k_titles) else "",
                    "search by vector": v_titles[i] if i < len(v_titles) else "",
                })
                
df = pd.DataFrame(structured_results)

[{'title': 'BBMRI-ERIC_FAQs_on_the_GDPR_V2.0-1.pdf', 'file_path': '../documents/GDPRAllerlei_903/BBMRI-ERIC_FAQs_on_the_GDPR_V2.0-1.pdf', 'content': 'Whether  data  qualifies  as \nanonymous  has to be established  on a case -by-case  basis,  requiring  risk assessment.  ‘Objective  factors’  \n(such  as the costs  and the amount  of time  required  ', 'score': 0.8563730535421705, 'search_type': 'llamaindex_bge'}, {'title': 'WP216_Opinion 05.2014 on Anonymisation Techniques.pdf', 'file_path': '../documents/WP29_270/WP216_Opinion 05.2014 on Anonymisation Techniques.pdf', 'content': 'This could also be presented the other way around: if a type of \ndata pr ocessing is not allowed under Article 6 of the e -Privacy Directive, there cannot be a \nlegal ground in Article 7 of the Data P', 'score': 0.8098895655474033, 'search_type': 'llamaindex_bge'}, {'title': 'eu-data-economy-legal-ethical--social-issues.pdf', 'file_path': '../documents/GDPRAllerlei_903/eu-data-economy-legal-ethical--social

/Users/tijsvandenheuvel/.virtualenvs/doc_retrieval_env/lib/python3.9/site-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  to

10 10
[{'title': '8.1 - Aanwijzing functionaris gegevensbescherming, Presentatie werking AVG.pdf', 'file_path': '../documents/GDPRAllerlei_903/8.1 - Aanwijzing functionaris gegevensbescherming, Presentatie werking AVG.pdf', 'content': 'Algemene Verordening Gegevensbescherming (AVG)\nWat verandert er allemaal ? AVG\nGenerieke wet\nalgemene kadersSpecifieke wetten\nWet RO\nDHW\nWmo2015\nJeugdwet\nParticipatiewet\nBRPKernbegrippen uit de AVG\n•P', 'score': 0.7973217766465971, 'search_type': 'llamaindex_bge'}, {'title': 'MinJenV - Handleiding AVG.pdf', 'file_path': '../documents/GDPRAllerlei_903/MinJenV - Handleiding AVG.pdf', 'content': 'Anonieme gegevens\nDe Verordening is niet van toepassing op anonieme gegevens, immers deze gegevens zijn niet terug te \nvoeren op een geïdentificeerde of identificeerbare natuurlijke persoon. Wanneer u', 'score': 0.7960417241735225, 'search_type': 'llamaindex_bge'}, {'title': 'advies_160_2018_0.pdf', 'file_path': '../documents/PrivCommGBA_GK_KC_409/advie

## Add to results file

In [10]:
print(df)

                                                 Query  ...                                  search by vector
0     When does aggregation of data qualify as anon...  ...                     MinJenV - Handleiding AVG.pdf
1                                                       ...  Weten of vergeten handreiking AVG Archiefwet.pdf
2                                                       ...       Nassima_ElKindi_Bachelorproef_18_19_pdf.pdf
3                                                       ...                  RUG01-002790173_2019_0001_AC.pdf
4                                                       ...                      gdpr-logboek-site-def189.pdf
..                                                 ...  ...                                               ...
575                                                     ...       Nassima_ElKindi_Bachelorproef_18_19_pdf.pdf
576                                                     ...              AVG-infoboekje-Privacy-op-School.pdf
577       

In [13]:
df = df.rename(columns={'search by keyword': 'search by BGE', 'search by vector': 'search by LABSE'})

In [14]:
print(df.iloc[1])

Query                                                               
search by BGE      WP216_Opinion 05.2014 on Anonymisation Techniq...
search by LABSE     Weten of vergeten handreiking AVG Archiefwet.pdf
Name: 1, dtype: object


In [19]:
bge = df['search by BGE']
labse = df['search by LABSE']

In [15]:
file_name2 = "../documents/20241029_Scenario_Results.xlsx"
df_existing_results = pd.read_excel(file_name2)

In [23]:
print(df_existing_results.iloc[1])

Query                                                              NaN
search by keyword         WP247_201704_wp247_en_ePrivacyRegulation.pdf
search by vector                              GDPR-factsheet-final.pdf
search by BGE        WP216_Opinion 05.2014 on Anonymisation Techniq...
search by LABSE       Weten of vergeten handreiking AVG Archiefwet.pdf
Name: 1, dtype: object


### Add new columns to existing results

In [20]:
df_existing_results = df_existing_results.join(bge)
df_existing_results = df_existing_results.join(labse)

## write to file

In [24]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

In [25]:
def write_to_file(df):
    # Write the DataFrame to an Excel file
    output_file = "../documents/20241029_Scenario_Results_new.xlsx"
    df.to_excel(output_file, index=False)
    
            # Apply styling using openpyxl
    workbook = load_workbook(output_file)
    sheet = workbook.active

    # Define zebra pattern fills
    fill1 = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")  # White
    fill2 = PatternFill(start_color="EAEAEA", end_color="EAEAEA", fill_type="solid")  # Light gray

    current_fill = fill1
    previous_query = None

    # Apply the zebra pattern
    for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
        query_cell = row[0]  # Query column          
        if query_cell.value :  # New query found
            if query_cell.value[0] != '(':
                current_fill = fill1 if current_fill == fill2 else fill2  # Alternate fills
                previous_query = query_cell.value

        # Apply the current fill to the entire row
        for cell in row:
            cell.fill = current_fill

    # Save the styled workbook
    workbook.save(output_file)

    print(f"Results successfully written to {output_file}")

In [26]:
write_to_file(df_existing_results)

Results successfully written to ../documents/20241029_Scenario_Results_new.xlsx
